# IPython Blockfolio

This is a simple "Blockfolio" using Python and the [CoinMarketCap API](https://coinmarketcap.com/api/).

It will show the net worth in USD of your Cryptocurrency investments as well as the relative share
of coins you own in comparison to the total coin supply of the corresponding digital asset.

First, the current exchanges rates are loaded, then combined with your investments, and finally displayed using Plotly.

This notebook can be found on [my github profile](https://github.com/SmokinCaterpillar/blockfolio).

Imports
--------

In [1]:
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

## Loading Data from CoinMarkteCap

In [2]:
limit = 200  # only load the top 200 currencies, if you have invested in smaller ones increase this limit
coin_market_cap_api = 'https://api.coinmarketcap.com/v1/ticker/?limit={}'.format(limit)
coin_market_cap_api

'https://api.coinmarketcap.com/v1/ticker/?limit=200'

In [3]:
# load the data using pandas
market_data = pd.read_json(coin_market_cap_api)
market_data.head(10)

,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,symbol,total_supply
0,1.126570e+10,16769287,bitcoin,1514541261,245574469614,2.100000e+07,Bitcoin,-1.49,1.00,7.10,1.000000,14644.300000,1,BTC,16769287
1,2.399560e+09,96639172,ethereum,1514541549,71365129254,NaN,Ethereum,-1.93,2.87,9.69,0.051166,738.470000,2,ETH,96639172
2,3.539290e+09,38739144847,ripple,1514541541,65190620340,1.000000e+11,Ripple,3.69,25.90,58.51,0.000117,1.682810,3,XRP,99993093880
3,3.534380e+09,16881963,bitcoin-cash,1514541553,41675656726,2.100000e+07,Bitcoin Cash,-2.36,-5.96,-0.61,0.171044,2468.650000,4,BCH,16881963
4,3.008320e+09,54522208,litecoin,1514541541,13264271844,8.400000e+07,Litecoin,-2.09,-4.00,-3.40,0.016856,243.282000,5,LTC,54522208
5,1.068760e+08,25927070538,cardano,1514541554,11478743793,4.500000e+10,Cardano,1.93,15.37,13.76,0.000031,0.442732,6,ADA,31112483745
6,1.911600e+08,2779530283,iota,1514541551,9898518834,2.779530e+09,IOTA,-4.18,2.91,-4.09,0.000247,3.561220,7,MIOTA,2779530283
7,1.609280e+08,7781857,dash,1514541541,8551015668,1.890000e+07,Dash,-2.19,0.40,-0.67,0.076135,1098.840000,8,DASH,7781857
8,4.747650e+07,8999999999,nem,1514541544,8218763999,NaN,NEM,-1.28,4.66,12.36,0.000063,0.913196,9,XEM,8999999999
9,1.365910e+08,15529227,monero,1514541542,5698915593,NaN,Monero,-1.21,-1.42,13.20,0.025427,366.980000,10,XMR,15529227


## Your Blockfolio

Below you can fill in the types and amount of coins you have into the Python dictionary
with the format `'SYMBOL': value`.

In [4]:
# enter your coins below:

block_folio = {
    'BTC': 0.31337,
    'BCH': 0.01337,
    'ETH': 3,
    'LTC': 1.5,
    'MIOTA': 777,
    'XRP': 13,
    'DASH': 0.1, 
    'XMR': 1, 
    'LSK': 12, 
    'OMG': 1.1, 
    'XRB': 42, 
    'PIVX': 123, 
    'ARK': 22, 
    'NEO': 7, 
    'GAS': 0.01,
    'STEEM': 499,
}

# display your blockfolio
block_folio = pd.DataFrame(list(block_folio.items()), columns=['symbol', 'amount'])
block_folio.head(len(block_folio))

,symbol,amount
0,PIVX,123.00000
1,STEEM,499.00000
2,ETH,3.00000
3,BCH,0.01337
4,ARK,22.00000
5,LTC,1.50000
6,XRB,42.00000
7,LSK,12.00000
8,OMG,1.10000
9,DASH,0.10000


### Combining the Blockfolio and the API Data

In [5]:
# merge the API and blockfolio data and sort by investment value
merged_data = block_folio.merge(market_data, how='left')
merged_data['value_usd'] = merged_data.amount * merged_data.price_usd
merged_data['coinshare'] = merged_data.amount / merged_data.available_supply
merged_data = merged_data.sort_values('value_usd', ascending=False)
merged_data.head()

,symbol,amount,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,total_supply,value_usd,coinshare
12,BTC,0.31337,1.126570e+10,16769287,bitcoin,1514541261,245574469614,2.100000e+07,Bitcoin,-1.49,1.00,7.10,1.000000,14644.30000,1,16769287,4589.084291,1.868714e-08
14,MIOTA,777.00000,1.911600e+08,2779530283,iota,1514541551,9898518834,2.779530e+09,IOTA,-4.18,2.91,-4.09,0.000247,3.56122,7,2779530283,2767.067940,2.795436e-07
2,ETH,3.00000,2.399560e+09,96639172,ethereum,1514541549,71365129254,NaN,Ethereum,-1.93,2.87,9.69,0.051166,738.47000,2,96639172,2215.410000,3.104331e-08
1,STEEM,499.00000,4.642830e+06,246344041,steem,1514541546,700331475,NaN,Steem,-1.66,-0.41,5.44,0.000197,2.84290,41,263318135,1418.607100,2.025622e-06
0,PIVX,123.00000,7.344400e+06,55233563,pivx,1514541546,606867730,NaN,PIVX,-1.17,-3.91,40.57,0.000761,10.98730,45,55233563,1351.437900,2.226907e-06


### Your Blockfolio in numbers and graphs!


In [6]:
networth = 'Your blockfolio is currently worth {:.2f} USD!'.format(merged_data.value_usd.sum())
print(networth)

Your blockfolio is currently worth 14763.89 USD!


#### Net Worth

Let's plot the worth of each asset in descending order:

In [7]:
marker = dict(color='rgb(158,202,225)',
              line=dict(color='rgb(8,48,107)', width=1.5))

layout = go.Layout(title='Blockfolio Networth in USD',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='USD'))
              
value_chart=go.Bar(x=merged_data.symbol, y=merged_data.value_usd, marker=marker)

fig = go.Figure(data=[value_chart], layout=layout)
py.iplot(fig)

#### Fraction of total Supply
Finally, that looks at the relative sizes of your investment. 
This gives you an idea or visualizes in which cryptocurrency you believe in the most. 
The graph below shows you fractional share of the the current crypto's total supply.

In [8]:
marker = dict(color='rgb(258,202,125)', line=dict(color='rgb(8,48,107)', width=1.5,))

layout = go.Layout(title='Relative Blockfolio Size',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='Fraction of total Supply'))

share_chart=go.Bar(x=merged_data.symbol, y=merged_data.coinshare, marker=marker)

fig = go.Figure(data=[share_chart], layout=layout)
py.iplot(fig)